### <font color='#4285f4'>Overview</font>

Overview: This notebook will generate Data for Order_header and order_detail table, code was generated using Gemini

Cost:

Approximate cost: $10

Author:

Navjot Singh

Adam Paternostro

### <font color='#4285f4'>License</font>

In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from datetime import datetime, timedelta, timezone
import random
from collections import defaultdict

# Create order_header Table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `agentic_beans_raw.order_header`(
      order_header_id INTEGER OPTIONS(description="Unique identifier for each order header."),
      order_header_timestamp TIMESTAMP OPTIONS(description="Timestamp when the order was placed."),
      customer_id INTEGER OPTIONS(description="Identifier for the customer who placed the order."),
      order_neighborhood STRING OPTIONS(description="Neighborhood where the order was placed or delivered, relevant for coffee truck operations.")
  )
  OPTIONS(
      description="Contains header information for orders placed at the coffee shop, potentially through coffee trucks."
  );

# Create order_detail Table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `agentic_beans_raw.order_detail` (
    order_detail_id INTEGER OPTIONS(description="Unique identifier for each line item in an order."),
    order_header_id INTEGER OPTIONS(description="Foreign key linking to the order_header table, indicating which order this detail belongs to."),
    truck_menu_id INTEGER OPTIONS(description="Identifier for the specific menu item ordered from the coffee truck's menu."),
    order_quantity INTEGER OPTIONS(description="Quantity of the specific menu item ordered.")
)
OPTIONS(
    description="Contains detailed line items for each order, including the menu item and quantity, relevant for coffee shop and truck operations."
);

# Order Header prompt we used with Gemini to Generate Python code for Data Generation

In [ ]:
order_header_prompt = """Give me python code to generate data for bigquery table order_header


Order table should have following columns


order_header_id - Primary Key with unique numeric value
order_header_timestamp - time the order was placed
customer_id - customer id of the customer who placed the order
order_neighborhood - location where order was placed


order_neighborhood should be one of the value from Select distinct weather_location from agentic_beans_raw.weather
customer_id should be randomly selected from customer_id column of the table agentic_beans_raw.customer
order_header_timestamp - it should be between Select max(observation_datetime), min(observation_datetime) from agentic_beans_raw.weather
order_header_timestamp should be between 6 AM to 10 PM for each day
No orders should be after 10 PM and before 6 AM
I’m looking for certain patterns for these orders, specifically the number of orders within one hour window
Number of orders within one hour window should depend on weather_description from agentic_beans_raw.weather table and events from agentic_beans_raw.events
If order_location has ‘Clear’ weather_description in weather_location in agentic_beans_raw.weather table for that hour which you can match using observation_datetime column with order_timestamp columns, you can have more orders for that hour for that location
Less random number of orders in case weather is Cloudy, Windy or Overcast
Number of orders for each hour should be cosine wave
If order_timestamp falls in between an event from events table using event_start_date_time
And event_end_date_time for the order_location matches to event_neighborhood in events table, then increase the number of orders in orders table for that event duration

Minimum orders within an hour window is 20 and maximum it could be 300
"""

# Order Detail prompt we used with Gemini to Generate Python code for Data Generation

In [ ]:
order_detail_prompt = """Give me python code to generate data for order_detail table



order_detail table should have following columns
order_detail_id - Primary key with unique numeric value
order_header_id - a key chosen from agentic_beans_raw.order table
truck_menu_id - truck_menu_id from agentic_beans_raw.truck_menu table
order_quantity - either 1 or 2 randomly


There could be 1 to 3 order details for one order_header_id

There are 20 truck ids in the agentic_beans_raw.truck_menu table

When choosing order_header_id from agentic_beans_raw.order_header table,  for a given hour in order_header_timestamp column,
place truck only in one order_neighborhood, and choose the truck_menu_id for that truck,
for example if on July 23rd 10 AM  truck id 3 is in Marble hill order_neighborhood,
it should choose truck_menu_id from agentic_beans_raw.truck_menu table where truck_id=3 for all the orders in that hour,
No other order_neighborhood should have menu Ids from truck_id=3

"""

**Once we got the Python Code from Gemini based on above prompts, we merged the code to generate data for both the tables, we are not going to regenerate the code again in this notebook**

# Define User Variables - Make Changes in this

In [ ]:
# --- 1. SCRIPT-WIDE CONFIGURATION ---

# --- User-defined variables ---
PROJECT_ID = "Add_your_project_id"  # <--- Change this to your GCP project ID
DATASET_ID = "agentic_beans_raw"
START_DATE_INPUT = "2025-07-28"  # <--- DEFINE the start date (inclusive)
END_DATE_INPUT = "2025-07-29"    # <--- DEFINE the end date (exclusive)

# --- Table IDs ---
ORDER_HEADER_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.order_header"
ORDER_DETAIL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.order_detail"
WEATHER_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.weather"
CUSTOMER_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.customer"
EVENTS_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.event"
TRUCK_MENU_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.truck_menu"

# BigQuery Helper Functions

In [ ]:
# --- 2. BIGQUERY HELPER FUNCTIONS ---

def get_max_id(client, table_id, column_name):
    """
    Queries a table to find the maximum existing ID in a specified column.
    Handles cases where the table is empty or doesn't exist.
    """
    print(f"Querying for MAX({column_name}) from {table_id}...")
    try:
        query = f"SELECT IFNULL(MAX({column_name}), 0) as max_id FROM `{table_id}`"
        results = client.query(query).result()
        row = next(results)
        max_id = row['max_id']
        print(f"Found max_id: {max_id}. New IDs will start from {max_id + 1}.")
        return max_id
    except Exception as e:
        print(f"WARNING: Could not query max ID from {table_id}. Assuming table is empty. Reason: {e}")
        return 0

def read_full_bigquery_table(client, table_id):
    """Reads an entire BigQuery table into a DataFrame."""
    print(f"Reading full table from {table_id}...")
    try:
        query = f"SELECT * FROM `{table_id}`"
        df = client.query(query).to_dataframe()
        print(f"Successfully read {len(df)} rows from {table_id}.")
        return df
    except Exception as e:
        print(f"ERROR: Failed to read {table_id}. Reason: {e}")
        return pd.DataFrame()

# Order header generation Function

In [ ]:
# --- 3. ORDER HEADER GENERATION LOGIC ---

def generate_order_header_for_month(client, month_start, month_end, locations, customer_ids, weather_df, event_lookup, start_order_id):
    """Generates order_header data for a single month."""
    print(f"\n--- Generating Order Headers for period: {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} ---")

    generated_orders = []
    order_id_counter = start_order_id
    min_orders_per_hour = 20
    max_orders_per_hour = 300

    day_iterator = month_start.date()
    while day_iterator <= month_end.date():
        for hour in range(6, 22):  # 6 AM to 10 PM
            hour_start_time = datetime.combine(day_iterator, datetime.min.time()).replace(hour=hour, tzinfo=timezone.utc)
            if not (month_start <= hour_start_time < month_end):
                continue

            hour_end_time = hour_start_time + timedelta(hours=1)

            for location in locations:
                cosine_adjustment = (np.cos((hour - 14) * np.pi / 8) + 1) / 2
                base_orders = int(min_orders_per_hour + (max_orders_per_hour - min_orders_per_hour) * cosine_adjustment)

                try:
                    weather_slice = weather_df.loc[hour_start_time:hour_end_time]
                    if not weather_slice.empty:
                        weather_now = weather_slice[weather_slice['weather_location'] == location]['weather_description'].iloc[0]
                        if weather_now == 'Clear':
                            base_orders *= 1.5
                        elif weather_now in ['Cloudy', 'Windy', 'Overcast']:
                            base_orders *= 0.7
                except (IndexError, KeyError):
                    pass

                is_event = False
                if location in event_lookup:
                    for start, end in event_lookup[location]:
                        if start < hour_end_time and end > hour_start_time:
                            is_event = True
                            break
                if is_event:
                    base_orders *= 4.0

                num_orders = int(np.clip(base_orders, min_orders_per_hour, max_orders_per_hour))

                for _ in range(num_orders):
                    order_time = hour_start_time + timedelta(seconds=random.randint(0, 3599))
                    if order_time < month_end:
                        generated_orders.append({
                            "order_header_id": order_id_counter,
                            "order_header_timestamp": order_time,
                            "Customer_id": random.choice(customer_ids),
                            "order_neighborhood": location
                        })
                        order_id_counter += 1

        day_iterator += timedelta(days=1)

    return pd.DataFrame(generated_orders)

# Order Detail Generation Function

In [ ]:
# --- 4. ORDER DETAIL GENERATION LOGIC ---

def generate_order_detail_data(orders_df, truck_menu_df, start_detail_id):
    """Generates order_detail data for a given DataFrame of orders."""
    print(f"--- Generating Order Details for the month ---")
    if orders_df.empty:
        print("No order headers to process, skipping detail generation.")
        return pd.DataFrame()

    orders_df['order_header_timestamp'] = pd.to_datetime(orders_df['order_header_timestamp'])
    orders_df['order_hour'] = orders_df['order_header_timestamp'].dt.floor('h')
    location_hours = orders_df[['order_neighborhood', 'order_hour']].drop_duplicates()
    all_truck_ids = truck_menu_df['truck_id'].unique()
    np.random.seed(42)
    location_hours['assigned_truck_id'] = np.random.choice(all_truck_ids, size=len(location_hours), replace=True)
    orders_df = pd.merge(
        orders_df,
        location_hours,
        on=['order_neighborhood', 'order_hour'],
        how='left'
    )

    num_orders = len(orders_df)
    orders_df['num_details'] = np.random.randint(1, 4, size=num_orders)
    order_detail_df = orders_df.loc[orders_df.index.repeat(orders_df['num_details'])].reset_index(drop=True)
    menu_by_truck = truck_menu_df.groupby('truck_id')['truck_menu_id'].apply(list)
    order_detail_df['possible_menus'] = order_detail_df['assigned_truck_id'].map(menu_by_truck)
    order_detail_df['truck_menu_id'] = [
        np.random.choice(menus) for menus in order_detail_df['possible_menus']
    ]
    order_detail_df['order_quantity'] = np.random.choice([1, 2], size=len(order_detail_df))

    num_new_details = len(order_detail_df)
    order_detail_df['order_detail_id'] = np.arange(start_detail_id, start_detail_id + num_new_details)

    final_df = order_detail_df[[
        'order_detail_id',
        'order_header_id',
        'truck_menu_id',
        'order_quantity'
    ]]

    return final_df

# Generate Order Header and Order Detail

In [ ]:
# --- 5. MAIN EXECUTION BLOCK ---

if __name__ == "__main__":
    # --- Initialization ---
    try:
        client = bigquery.Client(project=PROJECT_ID)
        print(f"BigQuery client initialized for project '{PROJECT_ID}'.")
    except Exception as e:
        exit(f"Failed to initialize BigQuery client. Error: {e}")

    try:
        # These are correctly created as timezone-aware
        start_date = pd.to_datetime(f'{START_DATE_INPUT} 00:00:00+0000', utc=True)
        end_date = pd.to_datetime(f'{END_DATE_INPUT} 00:00:00+0000', utc=True)
        monthly_periods = pd.period_range(start=start_date, end=end_date, freq='M')

    except ValueError as e:
        exit(f"Invalid START_DATE_INPUT or END_DATE_INPUT format. Please use 'YYYY-MM-DD'. Error: {e}")


    # --- Pre-fetch and Pre-process Data (Done Once) ---
    print("\n--- Pre-fetching required data ---")

    locations_df = read_full_bigquery_table(client, WEATHER_TABLE_ID)[['weather_location']].drop_duplicates()
    locations = locations_df['weather_location'].tolist()

    customers_df = read_full_bigquery_table(client, CUSTOMER_TABLE_ID)
    customer_ids = customers_df['customer_id'].tolist()

    truck_menu_df = read_full_bigquery_table(client, TRUCK_MENU_TABLE_ID)

    # Pre-process weather data
    print("Pre-processing weather data...")
    weather_df = read_full_bigquery_table(client, WEATHER_TABLE_ID)
    if not weather_df.empty:
        dt_col = pd.to_datetime(weather_df['observation_datetime'])
        if dt_col.dt.tz is None:
            print("Weather 'observation_datetime' is timezone-naive. Localizing to UTC.")
            weather_df['observation_datetime'] = dt_col.dt.tz_localize('UTC')
        else:
            print("Weather 'observation_datetime' is already timezone-aware. Converting to UTC.")
            weather_df['observation_datetime'] = dt_col.dt.tz_convert('UTC')
        weather_df = weather_df.set_index('observation_datetime').sort_index()

    # Pre-process events data
    print("Pre-processing events data...")
    events_df = read_full_bigquery_table(client, EVENTS_TABLE_ID)
    event_lookup = defaultdict(list)
    if not events_df.empty:
        for col_name in ['event_start_date_time', 'event_end_date_time']:
            dt_col = pd.to_datetime(events_df[col_name])
            if dt_col.dt.tz is None:
                print(f"Events '{col_name}' is timezone-naive. Localizing to UTC.")
                events_df[col_name] = dt_col.dt.tz_localize('UTC')
            else:
                print(f"Events '{col_name}' is already timezone-aware. Converting to UTC.")
                events_df[col_name] = dt_col.dt.tz_convert('UTC')
        for _, row in events_df.iterrows():
            event_lookup[row['event_neighborhood']].append(
                (row['event_start_date_time'], row['event_end_date_time'])
            )

    if not locations or not customer_ids or truck_menu_df.empty:
        exit("\nHalting execution: Could not read required lookup data (locations, customers, or truck menu).")


    # --- Initialize ID counters from BigQuery ---
    order_id_counter = get_max_id(client, ORDER_HEADER_TABLE_ID, 'order_header_id') + 1
    detail_id_counter = get_max_id(client, ORDER_DETAIL_TABLE_ID, 'order_detail_id') + 1

    total_headers_loaded = 0
    total_details_loaded = 0

    # --- Configure BigQuery Load Jobs (Done Once) ---
    header_job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("order_header_id", "INTEGER"),
            bigquery.SchemaField("order_header_timestamp", "TIMESTAMP"),
            bigquery.SchemaField("Customer_id", "INTEGER"),
            bigquery.SchemaField("order_neighborhood", "STRING"),
        ],
        write_disposition="WRITE_APPEND",
    )
    detail_job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("order_detail_id", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("order_header_id", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("truck_menu_id", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("order_quantity", "INTEGER", mode="REQUIRED"),
        ],
        write_disposition="WRITE_APPEND"
    )

    # --- Main Monthly Processing Loop ---
    for month_period in monthly_periods:
        # --- MODIFIED BLOCK (START) ---
        # FIX: Period.start_time and Period.end_time return NAIVE timestamps.
        # We must localize them to UTC to make them AWARE, so they can be
        # compared with the tz-aware start_date and end_date variables.
        month_start_timestamp = month_period.start_time.tz_localize('UTC')
        month_end_timestamp = month_period.end_time.tz_localize('UTC')
        # --- MODIFIED BLOCK (END) ---

        # Now all these variables are timezone-aware and can be compared safely.
        current_month_start = max(month_start_timestamp, start_date)
        current_month_end = min(month_end_timestamp, end_date)

        print(f"\n{'='*60}\nProcessing Period: {month_period} | From: {current_month_start.strftime('%Y-%m-%d %H:%M:%S %Z')} To: {current_month_end.strftime('%Y-%m-%d %H:%M:%S %Z')}\n{'='*60}")

        # --- STEP 1: Generate and Load Order Header Data ---
        order_header_df = generate_order_header_for_month(
            client, current_month_start, current_month_end, locations, customer_ids, weather_df, event_lookup, order_id_counter
        )

        if not order_header_df.empty:
            print(f"Generated {len(order_header_df)} order headers.")
            print(f"Uploading header batch to BigQuery table: {ORDER_HEADER_TABLE_ID}...")
            load_job = client.load_table_from_dataframe(
                order_header_df, ORDER_HEADER_TABLE_ID, job_config=header_job_config
            )
            load_job.result()
            headers_loaded = len(order_header_df)
            total_headers_loaded += headers_loaded
            order_id_counter += headers_loaded
            print(f"Successfully loaded {headers_loaded} header rows.")

            # --- STEP 2: Generate and Load Order Detail Data for the Same Month ---
            try:
                order_detail_df = generate_order_detail_data(
                    order_header_df, truck_menu_df, start_detail_id=detail_id_counter
                )

                if not order_detail_df.empty:
                    print(f"Generated {len(order_detail_df)} order details.")
                    print(f"Uploading detail batch to BigQuery table: {ORDER_DETAIL_TABLE_ID}...")
                    job = client.load_table_from_dataframe(
                        order_detail_df, ORDER_DETAIL_TABLE_ID, job_config=detail_job_config
                    )
                    job.result()
                    details_loaded = len(order_detail_df)
                    total_details_loaded += details_loaded
                    detail_id_counter += details_loaded
                    print(f"Successfully loaded {details_loaded} detail rows.")

            except KeyError as e:
                print(f"\nCRITICAL ERROR: A KeyError occurred during detail generation: {e}.")
                print("This likely means a column name in the script does not match your BigQuery table.")
                print("Halting further processing.")
                break
        else:
            print(f"No order headers were generated for the period {month_period}. Skipping to next period.")


    print(f"\n--- SCRIPT COMPLETE ---")
    print(f"Total Order Headers Loaded: {total_headers_loaded}")
    print(f"Total Order Details Loaded: {total_details_loaded}")